In [57]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import math

In [51]:
# log shadow model parameters
gamma = 2.1495
d0 = 40.0 # from drive tests
var = 100.0724  # from drive test 
# Lpld0 = 127.41
# Lpld0 = 98.0729
Lpld0 = 98.0729 + 7.5 # from drive test
print("PL0 (dBm) = ", Lpld0)
GL = 0

sensi125 = {
    7:-126.5,
    8:-127.25,
    9:-131.25,
    10:-132.75,
    11:-134.5,
    12:-133.25
}

bestVal = -134.5
sensi125_ideal = {
    7:bestVal+10,
    8:bestVal+7.5,
    9:bestVal+5.0,
    10:bestVal+2.5,
    11:bestVal,
    12:bestVal-2.5
}

bestVal = -128.75
sensi500_ideal = {
    7:bestVal+10,
    8:bestVal+7.5,
    9:bestVal+5.0,
    10:bestVal+2.5,
    11:bestVal,
    12:bestVal-2.5
}

print(sensi125)
# plt.figure()
# plt.plot(list(sensi125.keys()), list(sensi125.values()), "b.-")
# plt.plot(list(sensi125_ideal.keys()), list(sensi125_ideal.values()), "r.--")
# plt.grid()
# plt.show()
# sf7 = np.array([7,-126.5,-124.25,-120.75])
# sf8 = np.array([8,-127.25,-126.75,-124.0])
# sf9 = np.array([9,-131.25,-128.25,-127.5])
# sf10 = np.array([10,-132.75,-130.25,-128.75])
# sf11 = np.array([11,-134.5,-132.75,-128.75])
# sf12 = np.array([12,-133.25,-132.25,-132.25])

# # array of sensitivity values
# sensi = np.array([sf7,sf8,sf9,sf10,sf11,sf12])

def maxDist(PTX, sensi):
    dmax = d0*(10**((PTX-sensi-Lpld0)/(10*gamma)))
    return dmax

Ptx = 10  # in dBm
ranges125 = {}
for sf, sensi in sensi125_ideal.items():
    ranges125[sf] = maxDist(Ptx, sensi)

ranges500 = {}
for sf, sensi in sensi500_ideal.items():
    ranges500[sf] = maxDist(Ptx, sensi)

print("125 kHz:")
print(*ranges125.items(), sep="\n")
print("500 kHz:")
print(*ranges500.items(), sep="\n")

PL0 (dBm) =  105.5729
{7: -126.5, 8: -127.25, 9: -131.25, 10: -132.75, 11: -134.5, 12: -133.25}
125 kHz:
(7, 886.787311183723)
(8, 1159.1124963776126)
(9, 1515.0665354754824)
(10, 1980.3311706941358)
(11, 2588.474798825937)
(12, 3383.374398842821)
500 kHz:
(7, 478.9778161766544)
(8, 626.0680156518346)
(9, 818.328421451703)
(10, 1069.6304372910895)
(11, 1398.1052623711798)
(12, 1827.4520399965345)


In [73]:
# get table of supported devices
# what varies? SF and update rate
def loraSymboltime(bw, sf):
    return (2**sf)/bw

def airtime(sf,cr,pl,bw):
    H = 0        # implicit header disabled (H=0) or not (H=1)
    DE = 0       # low data rate optimization enabled (=1) or not (=0)
    Npream = 8   # number of preamble symbol (12.25  from Utz paper)

    if bw == 125 and sf in [11, 12]:
        # low data rate optimization mandated for BW125 with SF11 and SF12
        DE = 1
    if sf == 6:
        # can only have implicit header with SF6
        H = 1

    Tsym = (2.0**sf)/bw
    Tpream = (Npream + 4.25)*Tsym
    payloadSymbNB = 8 + max(np.ceil((8.0*pl-4.0*sf+28+16-20*H)/(4.0*(sf-2*DE)))*(cr+4),0)
    Tpayload = payloadSymbNB * Tsym
    return Tpream + Tpayload

def getMaxDev(rate, sfList, G=0.5, pl=20, bw=125e3, nCh=8):
    nmax = []
    for s in sfList:
        tPacket = airtime(sf=s, cr=1, pl=pl, bw=bw)
        n = G*nCh/(tPacket*rate)
        nmax.append(n)
    return nmax

sf = [7,8,9,10,11,12]
bw = 125e3  # in Hz
nCh = 8  # number of channels
packetLen = 20
times = [1, 10, 60, (60*5), (60*15), 3600]

for t in times:
    r = 1.0/t
    print("time {}, rate {}:".format(t, r))
    print("SF7, ... SF12, total")
    nmax = getMaxDev(r, sf, bw=bw, nCh=nCh, pl=packetLen)
    nmaxInt = np.array(np.round(nmax), dtype=int)
    tot = np.sum(nmaxInt)
    print(*nmaxInt, tot, sep=", ")
    print()

time 1, rate 1.0:
SF7, ... SF12, total
71, 39, 22, 11, 6, 3, 152

time 10, rate 0.1:
SF7, ... SF12, total
707, 389, 216, 108, 61, 30, 1511

time 60, rate 0.016666666666666666:
SF7, ... SF12, total
4242, 2332, 1295, 647, 364, 182, 9062

time 300, rate 0.0033333333333333335:
SF7, ... SF12, total
21210, 11660, 6474, 3237, 1820, 910, 45311

time 900, rate 0.0011111111111111111:
SF7, ... SF12, total
63631, 34981, 19423, 9712, 5459, 2730, 135936

time 3600, rate 0.0002777777777777778:
SF7, ... SF12, total
254525, 139925, 77693, 38847, 21836, 10918, 543744



In [74]:
print(10**(-2.5/(10*2)))

0.7498942093324559
